# Example 1: Relaxation of H2O vibrational state under polynomial PES

| run type      | wavefunction | backend | Basis  | steps |
| ---           | ---          | ---     | ---    | ---   |
| improved relaxation | MPS-SM | Numpy   | HO-DVR | 20    |

<img src="../pic/h2o.png" alt="h2o" width="200">

## 1. Import modules

- Required in **any** calculations

In [1]:
from pytdscf import BasInfo, Model, Simulator

## 2. Set primitive basis as HO-DVR
- FBR = Analytical integral of orthonormal basis
- DVR = Numerical integral of Localized orthonormal basis

|     | implemented | Supported operator | Supported backend |
| --- | ---         | ---                | --- |
| FBR | HO          | polynomial of Q and P (sum of product; SOP) | Numpy (recommended), JAX|
| DVR | HO, Sine    | arbitrary function (matrix product operator; MPO) | Numpy, JAX|

In [2]:
from math import sqrt

from discvar import HarmonicOscillator

from pytdscf import units
from pytdscf.potentials.h2o_potential import k_orig
from pytdscf.dvr_operator_cls import TensorOperator
from pytdscf.hamiltonian_cls import TensorHamiltonian

import numpy as np
import sympy
from pympo import AssignManager, OpSite, SumOfProducts

backend = "numpy"

freqs = [
    sqrt(k_orig[(1, 1)]),
    sqrt(k_orig[(2, 2)]),
    sqrt(k_orig[(3, 3)]),
] # a.u.
freqs

[0.007556564680635903, 0.017105918773130724, 0.017567584215986715]

In [3]:
nprims = [9, 9, 9]  # Number of primitive basis

basis = [
    HarmonicOscillator(nprim, omega, units="a.u.")
    for nprim, omega in zip(nprims, freqs, strict=False)
]
ndim = len(basis)  # Number of degree of freedom, H2O has 3 DOFs
basinfo = BasInfo([basis])  # Set basis information object

print(basinfo)

**MPS-MCTDH wavefunction**
$$
|\Psi_{\rm{MPS-MCTDH}}\rangle = \sum_{\mathbf \{j\}}\sum_{\mathbf \{\tau\}}
a\substack{j_1 \\ 1\tau_1}a\substack{j_2 \\ \tau_1\tau_2} \cdots a\substack{j_f \\ \tau_{f-1}1}
|\varphi_{j_1}^{(1)}(q_1)\rangle|\varphi_{j_2}^{(2)}(q_2)\rangle
\cdots|\varphi_{j_f}^{(f)}(q_f)\rangle
$$
where SPF is 
$$
\varphi_{j_p}^{(p)}(q_p) = \sum_{i_p=1}^{n_p} c_{i_p}^{j_p}\chi_{i_p}^{(p)}(q_p) \; (j_p = 1,2,\ldots, N_p)
$$

Here, select $\{\chi_{i_p}^{(p)}(q_p)\}$ as Harmonic Oscillator eigenfunction.
See detail in [documenation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.primints_cls.PrimBas_HO).
Here one defines $n_p$ = 9, $N_p$ = 9. (i.e., Standard Method)

**NOTE**

- In MPS,  $n = N$ (SM) is usually better than $n < M$ (MCTDH).  Only when using a laptop computer, MCTDH sometimes works better. (because the required RAM in MCTDH is smaller than SM.)

## 3. Set Hamiltonian (Polynomial Function)

- Here, one uses pre-calculated Polyonimal PES.
- `k_orig` is the coefficients of Taylor expansion of PES in reference geometry by mass-weighted coordinate $Q_i$ (The unit of coordinate is **NOT AMU** but $\sqrt{m_e}a_0$, i.e. atomic units)
$$ V - V_0 = \frac{k_{11}}{2!} Q_1^2 + \frac{k_{22}}{2!} Q_2^2 + \frac{k_{33}}{2!} Q_3^2 + \frac{k_{111}}{3!} Q_1^3 + \frac{k_{122}}{2!} Q_1Q_2^2 + \cdots $$

In [4]:
k_orig

defaultdict(float,
            {(1, 1): 5.7101669772633975e-05,
             (2, 2): 0.00029261245707294615,
             (3, 3): 0.0003086200151857856,
             (1, 1, 1): -8.973542624563865e-07,
             (2, 2, 2): -1.8571147341445975e-05,
             (1, 2, 2): 5.028987089424822e-07,
             (1, 1, 2): 1.2870557913839666e-06,
             (1, 3, 3): 2.0063268625796784e-06,
             (2, 3, 3): -1.8853947560756764e-05,
             (1, 1, 1, 1): -2.2778131948543168e-08,
             (2, 2, 2, 2): 1.042951948572713e-06,
             (3, 3, 3, 3): 1.1133748664915738e-06,
             (1, 2, 2, 2): -8.193988329963448e-08,
             (1, 1, 2, 2): -1.852073688081903e-07,
             (1, 1, 1, 2): 5.750959195886642e-08,
             (1, 1, 3, 3): -2.1211138514059556e-07,
             (2, 2, 3, 3): 1.0721581542221527e-06,
             (1, 2, 3, 3): -1.256574051408931e-07})

### Setup one particle operator

In [5]:
q1_list = [np.array(basis_i.get_grids()) for basis_i in basis]
q2_list = [q1_int**2 for q1_int in q1_list]
q3_list = [q1_int**3 for q1_int in q1_list]
q4_list = [q1_int**4 for q1_int in q1_list]
q0_list = [q1_int**0 for q1_int in q1_list]
p2_list = [basis_i.get_2nd_derivative_matrix_dvr() for basis_i in basis]

p2_ops = [OpSite(f"p^2_{i+1}", i, value=p2_list[i]) for i in range(0, ndim)]
q4_ops = [OpSite(f"q^4_{i+1}", i, value=q4_list[i]) for i in range(0, ndim)]
q3_ops = [OpSite(f"q^3_{i+1}", i, value=q3_list[i]) for i in range(0, ndim)]
q2_ops = [OpSite(f"q^2_{i+1}", i, value=q2_list[i]) for i in range(0, ndim)]
q1_ops = [OpSite(f"q_{i+1}", i, value=q1_list[i]) for i in range(0, ndim)]
q0_ops = [OpSite(f"q^0_{i+1}", i, value=q0_list[i]) for i in range(0, ndim)]

qn_list = [q0_ops, q1_ops, q2_ops, q3_ops, q4_ops]

In [6]:
from collections import Counter

k_tensor = np.zeros((5, 5, 5))
key_list = list(k_orig.keys())

for key in key_list:
    count = Counter(key)
    k_tensor[count[1]][count[2]][count[3]] = k_orig[key]

k_symbol = [[[sympy.symbols("k_{" + "1" * i + "2" * j + "3" * k + "}") for k in range(5)]
                                     for j in range(5)]
                                     for i in range(5)]
subs = {}

for i in range(5):
    for j in range(5):
        for k in range(5):
            subs[k_symbol[i][j][k]] = k_tensor[i][j][k]

### Setup potential and kinetic operator

In [7]:
from scipy.special import factorial

pot_sop = SumOfProducts()

for i in range(3):
    pot_sop += -0.5 * p2_ops[i]

for i in range(5):
    for j in range(5):
        for k in range(5):
            if i == 0 and j == 0 and k == 0:
                continue
            if k_tensor[i][j][k] == 0:
                continue
            pot_sop += k_symbol[i][j][k] * qn_list[i][0] * qn_list[j][1] * qn_list[k][2] / factorial(i) / factorial(j) / factorial(k)

pot_sop = pot_sop.simplify()
pot_sop.symbol

0.0416666666666667*k_{1111}*q^4_1*q^0_2*q^0_3 + 0.166666666666667*k_{1112}*q^3_1*q_2*q^0_3 + 0.166666666666667*k_{111}*q^3_1*q^0_2*q^0_3 + 0.25*k_{1122}*q^2_1*q^2_2*q^0_3 + 0.5*k_{112}*q^2_1*q_2*q^0_3 + 0.25*k_{1133}*q^2_1*q^0_2*q^2_3 + 0.5*k_{11}*q^2_1*q^0_2*q^0_3 + 0.166666666666667*k_{1222}*q_1*q^3_2*q^0_3 + 0.5*k_{122}*q_1*q^2_2*q^0_3 + 0.5*k_{1233}*q_1*q_2*q^2_3 + 0.5*k_{133}*q_1*q^0_2*q^2_3 + 0.0416666666666667*k_{2222}*q^0_1*q^4_2*q^0_3 + 0.166666666666667*k_{222}*q^0_1*q^3_2*q^0_3 + 0.25*k_{2233}*q^0_1*q^2_2*q^2_3 + 0.5*k_{22}*q^0_1*q^2_2*q^0_3 + 0.5*k_{233}*q^0_1*q_2*q^2_3 + 0.0416666666666667*k_{3333}*q^0_1*q^0_2*q^4_3 + 0.5*k_{33}*q^0_1*q^0_2*q^2_3 - 0.5*p^2_1 - 0.5*p^2_2 - 0.5*p^2_3

### Setup MPO

In [8]:
am_pot = AssignManager(pot_sop)
am_pot.assign()
display(*am_pot.Wsym)
W_prod = sympy.Mul(*am_pot.Wsym)
print(*[f"W{i}" for i in range(am_pot.ndim)], "=")
display(W_prod[0].expand())
pot_mpo = am_pot.numerical_mpo(subs=subs)

2025-04-30 12:17:59.226 | INFO     | pympo.bipartite:assign:286 - assigned 1/3
2025-04-30 12:17:59.245 | INFO     | pympo.bipartite:assign:286 - assigned 2/3
2025-04-30 12:17:59.253 | INFO     | pympo.bipartite:assign:286 - assigned 3/3


Matrix([[q^4_1, q_1, p^2_1, 1, q^2_1, q^3_1, q^0_1]])

Matrix([
[         0,                                                        0,                                                      0.0416666666666667*k_{1111}*q^0_2, 0,     0],
[         0,                     0.5*k_{1233}*q_2 + 0.5*k_{133}*q^0_2,                                   0.166666666666667*k_{1222}*q^3_2 + 0.5*k_{122}*q^2_2, 0,     0],
[      -0.5,                                                        0,                                                                                      0, 0,     0],
[-0.5*p^2_2,                                                        0,                                                                                      0, 1,     0],
[         0,                                      0.25*k_{1133}*q^0_2,                               0.25*k_{1122}*q^2_2 + 0.5*k_{112}*q_2 + 0.5*k_{11}*q^0_2, 0,     0],
[         0,                                                        0,                       0.166666666666667*k_{1112}*q_2 + 0.1666666666666

Matrix([
[                                1],
[                            q^2_3],
[                            q^0_3],
[                       -0.5*p^2_3],
[0.0416666666666667*k_{3333}*q^4_3]])

W0 W1 W2 =


0.0416666666666667*k_{1111}*q^4_1*q^0_2*q^0_3 + 0.166666666666667*k_{1112}*q^3_1*q_2*q^0_3 + 0.166666666666667*k_{111}*q^3_1*q^0_2*q^0_3 + 0.25*k_{1122}*q^2_1*q^2_2*q^0_3 + 0.5*k_{112}*q^2_1*q_2*q^0_3 + 0.25*k_{1133}*q^2_1*q^0_2*q^2_3 + 0.5*k_{11}*q^2_1*q^0_2*q^0_3 + 0.166666666666667*k_{1222}*q_1*q^3_2*q^0_3 + 0.5*k_{122}*q_1*q^2_2*q^0_3 + 0.5*k_{1233}*q_1*q_2*q^2_3 + 0.5*k_{133}*q_1*q^0_2*q^2_3 + 0.0416666666666667*k_{2222}*q^0_1*q^4_2*q^0_3 + 0.166666666666667*k_{222}*q^0_1*q^3_2*q^0_3 + 0.25*k_{2233}*q^0_1*q^2_2*q^2_3 + 0.5*k_{22}*q^0_1*q^2_2*q^0_3 + 0.5*k_{233}*q^0_1*q_2*q^2_3 + 0.0416666666666667*k_{3333}*q^0_1*q^0_2*q^4_3 + 0.5*k_{33}*q^0_1*q^0_2*q^2_3 - 0.5*p^2_1 - 0.5*p^2_2 - 0.5*p^2_3

### Setup Hamiltonian

In [9]:
potential = [
    [
        {
            (tuple((i, i) for i in range(0, ndim))): TensorOperator(
                mpo=pot_mpo
            )
        }
    ]
]

H = TensorHamiltonian(
    ndof=len(basis), potential=potential, kinetic=None, backend=backend
)

operators = {"hamiltonian": H}

## 4. Set wavefunction (MPS) and All Model

- `m_aux_max` is a bond dimension of MPS (maximum rank of auxiliary index $\tau_p$)



In [10]:
model = Model(basinfo, operators)
model.m_aux_max = 9

## 5. Execute Calculation

- time step width is defined by `stepsize`=0.1 fs

F.Y.I., See also documentation about [Simulator](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.simulator_cls.Simulator)

**NOTE**

- Runtype cannot be rebound. If you change the runtype, you should restart the kernel.

- Improved relaxation, i.e., diagonalization-based variational optimization, is much faster than pure imaginary time evolution.

- When simulating larger systems, (f>6, m>10), contraction of tensors can be overhead, in such a situation JAX, especially supporting GPU, is recommended to use as a backend.

In [11]:
jobname = "h2o_polynomial"
simulator = Simulator(jobname, model, ci_type="MPS", backend="Numpy", verbose=4)
simulator.relax(savefile_ext="_gs", maxstep=20, stepsize=0.1)

2025-04-30 12:18:00,132 - INFO:main.pytdscf._const_cls - 
     ____     __________   .____ ____   _____
    / _  |   /__  __/ _ \ / ___ / _  \ / ___/
   / /_) /_  __/ / / / ||/ /__ / / )_// /__
  /  ___/ / / / / / / / |.__  / |  __/ ___/
 /  /  / /_/ / / / /_/ /___/ /| \_/ / /
/__/   \__, /_/ /_____/_____/ \____/_/
      /____/

2025-04-30 12:18:00,134 - INFO:main.pytdscf._const_cls - Log file is ./h2o_polynomial_relax/main.log
2025-04-30 12:18:00,136 - INFO:main.pytdscf.simulator_cls - Set integral of DVR basis
2025-04-30 12:18:00,152 - INFO:main.pytdscf.simulator_cls - Set initial wave function (DVR basis)
2025-04-30 12:18:00,154 - INFO:main.pytdscf.simulator_cls - Prepare MPS w.f.
2025-04-30 12:18:00,156 - INFO:main.pytdscf._mps_cls - Initial MPS: 0-state with weights 1.0
2025-04-30 12:18:00,158 - INFO:main.pytdscf._mps_cls - Initial MPS: 0-state 0-mode with weight [1. 0. 0. 0. 0. 0. 0. 0. 0.]
2025-04-30 12:18:00,160 - INFO:main.pytdscf._mps_cls - Initial MPS: 0-state 1-mode with we

(0.020855716347418743, <pytdscf.wavefunction.WFunc at 0x1222c4980>)

## 6. Check Log file
See `h2o_polynomial_relax/main.log`, which is defined in `jobname`

In [12]:
!tail h2o_polynomial_relax/main.log

/usr/local/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.400 | elapsed[sec]:     0.62 | ci:  0.6  (ci_exp:  0.4|ci_rnm:  0.2|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.500 | elapsed[sec]:     0.64 | ci:  0.6  (ci_exp:  0.4|ci_rnm:  0.2|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.600 | elapsed[sec]:     0.66 | ci:  0.7  (ci_exp:  0.4|ci_rnm:  0.2|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.700 | elapsed[sec]:     0.69 | ci:  0.7  (ci_exp:  0.4|ci_rnm:  0.2|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.800 | elapsed[sec]:     0.71 | ci:  0.7  (ci_exp:  0.5|ci_rnm:  0.2|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
Saved wavefunction    1.900 [fs]
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.900 | elapsed[sec]:     0.73 | ci:  0.7  (ci_exp:  0.5|ci_rnm:  0.3|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
E

**Vibrational ground state energy is found to be `0.5675130` eV!**